In [31]:
import re

# Function to clean file names
def clean_file_name(file_name):
    # Remove invalid characters from the file name
    cleaned_name = re.sub(r'[<>:"/\\|?*]', '', file_name)
    return cleaned_name

In [39]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from mutagen.id3 import ID3, TIT2, TPE1, TALB

# Set up your Spotify API credentials
client_id = 'f1ef4b12e36144c5a677c480f197730a'
client_secret = 'b3783943d8b84273a8817e5f94ee234d'

# Set up the Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to update metadata
def update_metadata(file_path):
    # Extract song name from the file name
    song_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Search for the song using the Spotify API
    results = sp.search(q=song_name, limit=1, type='track')
    
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        
        # Extract the relevant metadata
        title = track['name']
        artist = track['artists'][0]['name']
        album = track['album']['name']
        
        # Update the metadata of the MP3 file
        try:
            audio = ID3(file_path)
            audio['TIT2'] = TIT2(encoding=3, text=title)
            audio['TPE1'] = TPE1(encoding=3, text=artist)
            audio['TALB'] = TALB(encoding=3, text=album)
            audio.save()
        except:
            print(f"No results found for '{song_name}'.")
        
        # Generate the new file name
        new_file_name = f"{artist} - {title}.mp3"
        new_file_name = clean_file_name(new_file_name)  # Clean the new file name
        new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
        
        # Check if the new file path already exists
        counter = 1
        while os.path.exists(new_file_path):
            file_name, file_extension = os.path.splitext(new_file_name)
            new_file_name = f"{file_name} ({counter}){file_extension}"
            new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
            counter += 1

        # Rename the file
        os.rename(file_path, new_file_path)

        # Store the original and new file names
        with open('file_names_mapping.txt', 'a') as file:
            file.write(f"{file_name},{new_file_name}\n")

    else:
        print(f"No results found for '{song_name}'.")

# Specify the root directory where your MP3 files are located
root_directory = 'D:\Music'

# Recursively traverse the directory and update metadata and file names
for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith('.mp3'):
            file_path = os.path.join(dirpath, filename)
            update_metadata(file_path)


No results found for 'The Script - Before the Worst'.
No results found for 'The Script - We Cry'.


In [40]:
# Undo the changes and revert the file names
def undo_changes():
    # Read the file_names_mapping.txt to get the original and new file names
    with open('file_names_mapping.txt', 'r') as file:
        mappings = file.readlines()

    # Iterate through the mappings and revert the file names
    for mapping in mappings:
        mapping = mapping.strip().split(',')
        original_file_name, new_file_name = mapping[0], mapping[1]
        original_file_path = os.path.join(root_directory, original_file_name)
        new_file_path = os.path.join(root_directory, new_file_name)

        # Rename the file back to the original file name
        os.rename(new_file_path, original_file_path)

    # Remove the file_names_mapping.txt file
    os.remove('file_names_mapping.txt')